# Quickstart

We begin with a straightforward example: PCA (EOF analysis) of a 3D `xarray.Dataset`.

## Import the package
We start by importing the xarray and xeofs package.

In [ ]:
import xarray as xr
import xeofs as xe

xr.set_options(display_expand_attrs=False)

## Load the data
Next, we fetch the data from the xarray tutorial repository. The data is a 3D dataset of 6 hourly surface air temperature over North America between 2013 and 2014. 

In [ ]:
t2m = xr.tutorial.open_dataset("air_temperature")
t2m

## Fit the model

To apply PCA to the data, we first need to create an `EOF` object. Since our data is on a regular latitude-longitude grid, we need to weigh each grid cell by the square root of the cosine of the latitude. This can be enabled by setting `use_coslat=True`.

In [ ]:
model = xe.single.EOF(use_coslat=True)

We will now fit the model to the data. If you've worked with scikit-learn before, this process will seem familiar. However, there's an important difference: while scikit-learn's `fit` method typically assumes 2D input data shaped as (`sample` x `feature`), our scenario is less straightforward. For any model, including PCA, we must specify the sample dimension. With this information, xeofs will interpret all other dimensions as feature dimensions (more on that [here](core_functionalities/labeled_data.rst)).

In climate science, it's common to maximize variance along the time dimension when applying PCA. Yet, this isn't the sole approach. For instance, [Compagnucci & Richmann (2007)](https://rmets.onlinelibrary.wiley.com/doi/abs/10.1002/joc.1574) discuss alternative applications. `xeofs` offers flexibility in this aspect. You can designate multiple sample dimensions, provided at least one feature dimension remains. For our purposes, we'll set `time` as our sample dimension and then fit the model to our data.

In [ ]:
model.fit(t2m, dim="time")

## Inspect the results

Now that the model has been fitted, we can examine the result. For example, one typically starts by looking at the explained variance (ratio). 

In [ ]:
model.explained_variance_ratio()

We can next examine the spatial patterns, which are the eigenvectors of the covariance matrix, often referred to as EOFs or principal components.

> **_NOTE:_** The xeofs library aims to adhere to the convention where the primary patterns obtained from dimensionality reduction (which typically exclude the sample dimension) are termed components (akin to principal components). When data is projected onto these patterns, for instance using the `transform` method, the outcome is termed `scores` (similar to principal component scores). However, this terminology is more of a guideline than a strict rule.

In [ ]:
components = model.components()
components

You'll observe that the result is an `xr.Dataset`, mirroring the format of our original input data. To visualize the components, we can use typical methods for xarray objects. Now, let's inspect the first component.

> **_NOTE:_**  xeofs is designed to match the data type of its input. For instance, if you provide an `xr.DataArray` as input, the components will also be of type `xr.DataArray`. Similarly, if the input is an `xr.Dataset`, the components will mirror that as an `xr.Dataset`. The same principle applies if the input is a `list`; the output components will be presented in a `list` format. This consistent behavior is maintained across all xeofs methods.

In [ ]:
components["air"].sel(mode=1).plot()

We can also examine the principal component scores, which represent the corresponding time series.

> **_NOTE:_**  When comparing the scores from xeofs to outputs from other PCA implementations like [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) or [eofs](https://ajdawson.github.io/eofs/latest/), you might spot discrepancies in the absolute values. This arises because xeofs typically returns scores normalized by the L2 norm. However, if you prefer unnormalized scores, simply set `normalized=False` when using the `scores` or `transform` method."

In [ ]:
model.scores().sel(mode=1).plot()